In [17]:
import json
from jsonpath import jsonpath
import requests
import pandas as pd

In [18]:
# 通过爬虫获取 JSON 格式文本
# 方法一：使用 json.loads() 方法 读取 JSON 格式文本
url = 'http://www.shfe.com.cn/data/dailydata/kx/kx20210219.dat'
res = requests.get(url)
json_data = json.loads(res.text)
json_data

{'o_curinstrument': [{'PRODUCTID': 'cu_f    ',
   'PRODUCTGROUPID': 'cu      ',
   'PRODUCTSORTNO': 10,
   'PRODUCTNAME': '铜                  ',
   'DELIVERYMONTH': '2103',
   'PRESETTLEMENTPRICE': 62600,
   'OPENPRICE': 63440,
   'HIGHESTPRICE': 64330,
   'LOWESTPRICE': 62890,
   'CLOSEPRICE': 64170,
   'SETTLEMENTPRICE': 63390,
   'ZD1_CHG': 1570,
   'ZD2_CHG': 790,
   'VOLUME': 106671,
   'TASVOLUME': '',
   'OPENINTEREST': 68887,
   'OPENINTERESTCHG': -11804,
   'ORDERNO': 0,
   'ORDERNO2': 0},
  {'PRODUCTID': 'cu_f    ',
   'PRODUCTGROUPID': 'cu      ',
   'PRODUCTSORTNO': 10,
   'PRODUCTNAME': '铜                  ',
   'DELIVERYMONTH': '2104',
   'PRESETTLEMENTPRICE': 62730,
   'OPENPRICE': 63390,
   'HIGHESTPRICE': 64470,
   'LOWESTPRICE': 63000,
   'CLOSEPRICE': 64320,
   'SETTLEMENTPRICE': 63560,
   'ZD1_CHG': 1590,
   'ZD2_CHG': 830,
   'VOLUME': 199534,
   'TASVOLUME': '',
   'OPENINTEREST': 147830,
   'OPENINTERESTCHG': 19979,
   'ORDERNO': 0,
   'ORDERNO2': 0},
  {'PRODUCT

In [19]:
# 使用 json 库读取 JSON 内容
datas_heyue = []
results_heyue = json_data['o_curinstrument']
for heyue in results_heyue:
    data_heyue = {
                '日期': '20220219',
                '交易所': 'SQH',
                '商品名称': heyue['PRODUCTNAME'].strip(),
                '正常成交量': heyue['VOLUME'],
                'TAS成交量': heyue['TASVOLUME'],
                '持仓量': heyue['OPENINTEREST'],
                '收盘价': heyue['CLOSEPRICE'],
                '备注':heyue['DELIVERYMONTH'].strip()
                }
    datas_heyue.append(data_heyue)
df_heyue = pd.DataFrame(datas_heyue)
df_heyue

,日期,交易所,商品名称,正常成交量,TAS成交量,持仓量,收盘价,备注
0,20220219,SQH,铜,106671,,68887,64170,2103
1,20220219,SQH,铜,199534,,147830,64320,2104
2,20220219,SQH,铜,69650,,75898,64370,2105
3,20220219,SQH,铜,22045,,32707,64390,2106
4,20220219,SQH,铜,5058,,11851,64330,2107
...,...,...,...,...,...,...,...,...
260,20220219,SQH,纸浆,2,,9,6402,2202
261,20220219,SQH,纸浆,554648,,421391,,小计
262,20220219,SQH,总计,9099800,0,6762240,,
263,20220219,SQH,原油TAS,,0,,,2104


In [20]:
# 方法二：使用 pandas 库的 json_normalize() 方法读取多层嵌套 JSON 格式文本
df_json = pd.json_normalize(json_data, ['o_curinstrument'],max_level=1)
df_json

,PRODUCTID,PRODUCTGROUPID,PRODUCTSORTNO,PRODUCTNAME,DELIVERYMONTH,PRESETTLEMENTPRICE,OPENPRICE,HIGHESTPRICE,LOWESTPRICE,CLOSEPRICE,SETTLEMENTPRICE,ZD1_CHG,ZD2_CHG,VOLUME,TASVOLUME,OPENINTEREST,OPENINTERESTCHG,ORDERNO,ORDERNO2
0,cu_f,cu,10,铜,2103,62600,63440,64330,62890,64170,63390,1570,790,106671,,68887,-11804,0,0
1,cu_f,cu,10,铜,2104,62730,63390,64470,63000,64320,63560,1590,830,199534,,147830,19979,0,0
2,cu_f,cu,10,铜,2105,62710,63480,64510,63030,64370,63600,1660,890,69650,,75898,5289,0,0
3,cu_f,cu,10,铜,2106,62640,63400,64470,63000,64390,63620,1750,980,22045,,32707,3916,0,0
4,cu_f,cu,10,铜,2107,62700,63350,64420,62950,64330,63680,1630,980,5058,,11851,1791,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,sp_f,sp,110,纸浆,2202,6390,6402,6402,6402,6402,6402,12,12,2,,9,0,0,0
261,sp_f,sp,110,纸浆,小计,,,,,,,,,554648,,421391,28567,0,50
262,总计,,9999,总计,,,,,,,,,,9099800,0,6762240,288493,0,0
263,sc_tas,sc_tas,90,原油TAS,2104,,,,,,,,,,0,,,999,0


In [21]:
# 方法三：使用 jsonpath 库读取 JSON 格式文本
product_name = jsonpath(json_data, '$..PRODUCTNAME')
volume = jsonpath(json_data, '$..VOLUME')
TAS_volume = jsonpath(json_data, '$..TASVOLUME')
open_interest = jsonpath(json_data, '$..OPENINTEREST')
close_price = jsonpath(json_data, '$..CLOSEPRICE')
delivery_month = jsonpath(json_data, '$..DELIVERYMONTH')
list_content = [product_name,volume,TAS_volume,open_interest,close_price,delivery_month]
df_jsonpath = pd.DataFrame(list_content)
df_jsonpath = df_jsonpath.T
df_jsonpath.rename(columns={0:'product_name',1:'volume',2:'TAS_volume',3:'open_interest',4:'close_price',5:'delivery_month'},inplace=True)
df_jsonpath = df_jsonpath.applymap(str) # 将所有元素转换为文本
df_jsonpath = df_jsonpath.applymap(lambda x:x.replace('None','')) # 替换内容
df_jsonpath

,product_name,volume,TAS_volume,open_interest,close_price,delivery_month
0,铜,106671,,68887,64170,2103
1,铜,199534,,147830,64320,2104
2,铜,69650,,75898,64370,2105
3,铜,22045,,32707,64390,2106
4,铜,5058,,11851,64330,2107
...,...,...,...,...,...,...
281,石油沥青,565981,,,,
282,天然橡胶,668811,,,,
283,20号胶,59593,,,,
284,纸浆,554648,,,,


In [22]:
# 写入生成 JSON 文件
# 方法一：使用 json.dumps() 方法生成 JSON 文件
json_text = json.dumps(json_data, sort_keys=True, indent=4, ensure_ascii=False) # 参数设置为 ensure_ascii=False，解决中文显示乱码问题
with open('./json_text.json', 'w') as f:
    f.write(json_text)
json_text

'{\n    "o_IMChangeDate": "20150813",\n    "o_code": 0,\n    "o_curinstrument": [\n        {\n            "CLOSEPRICE": 64170,\n            "DELIVERYMONTH": "2103",\n            "HIGHESTPRICE": 64330,\n            "LOWESTPRICE": 62890,\n            "OPENINTEREST": 68887,\n            "OPENINTERESTCHG": -11804,\n            "OPENPRICE": 63440,\n            "ORDERNO": 0,\n            "ORDERNO2": 0,\n            "PRESETTLEMENTPRICE": 62600,\n            "PRODUCTGROUPID": "cu      ",\n            "PRODUCTID": "cu_f    ",\n            "PRODUCTNAME": "铜                  ",\n            "PRODUCTSORTNO": 10,\n            "SETTLEMENTPRICE": 63390,\n            "TASVOLUME": "",\n            "VOLUME": 106671,\n            "ZD1_CHG": 1570,\n            "ZD2_CHG": 790\n        },\n        {\n            "CLOSEPRICE": 64320,\n            "DELIVERYMONTH": "2104",\n            "HIGHESTPRICE": 64470,\n            "LOWESTPRICE": 63000,\n            "OPENINTEREST": 147830,\n            "OPENINTERESTCHG":

In [23]:
# 方法二：使用 pandas 库的 to_json() 方法生成 JSON 文件
json_pd = df_jsonpath.to_json(orient='table',force_ascii=False) # 参数设置为 force_ascii=False，解决中文显示乱码问题
with open('./json_pd.json', 'w') as f:
    f.write(json_pd)
json_pd

'{"schema":{"fields":[{"name":"index","type":"integer"},{"name":"product_name","type":"string"},{"name":"volume","type":"string"},{"name":"TAS_volume","type":"string"},{"name":"open_interest","type":"string"},{"name":"close_price","type":"string"},{"name":"delivery_month","type":"string"}],"primaryKey":["index"],"pandas_version":"0.20.0"},"data":[{"index":0,"product_name":"铜                  ","volume":"106671","TAS_volume":"","open_interest":"68887","close_price":"64170","delivery_month":"2103"},{"index":1,"product_name":"铜                  ","volume":"199534","TAS_volume":"","open_interest":"147830","close_price":"64320","delivery_month":"2104"},{"index":2,"product_name":"铜                  ","volume":"69650","TAS_volume":"","open_interest":"75898","close_price":"64370","delivery_month":"2105"},{"index":3,"product_name":"铜                  ","volume":"22045","TAS_volume":"","open_interest":"32707","close_price":"64390","delivery_month":"2106"},{"index":4,"product_name":"铜            